# D3b Data Engineer Exercises

There are two sections, the first is composed of
straight coding exercises and the second are exercises based on a provided
SQLite database. Each section shouldn’t take more than about an hour, but we
provide 24 hours so that you have time to provide code that you’re happy with.
You can email your solution or provide a pointer to it hosted somewhere like
GitHub/GitLab.

## Coding Section

Write a solution for the functions/classes
described below, please consider:

  - Efficiency in both space and time
  -
Edge cases and error handling
  - Tests

We prefer you to use python. You can
use other languages, but please explain why. 

### Coding Exercise 1: Flatten
Nested List

In [1]:
def flatten_list(nested_list):
    """
    Flatten an
arbitrarily nested list

    Parameters
    ----------
    nested_list : nested list of int List with item to be either integers or list
    Example: [2,[[3,[4]], 5]]

    Returns
    -------
    flat_list : list of int
    A flattened list with only integers
        Example: [2,3,4,5]
      """

In [2]:
from collections import abc, namedtuple
from itertools import chain

def flatten_list(nested_list):
    "returns an iterable of each integer in arbitrarily nested list.  use list() to collect"
    return chain.from_iterable(
            flatten_list(elem) if isinstance(elem, abc.Iterable) else [elem]
            for elem in nested_list)

In [3]:
list(flatten_list([0, [1, [5, 4, 3], 3], [4]]))

[0, 1, 5, 4, 3, 3, 4]

### Coding Exercise 2: Serialize a Binary Tree

Serializes a tree
from top to
bottom, left to right to a list of values
Parameters
----------
root_node : root
node of a binary tree
    The tree is not
ordered or balanced, it's just a
binary tree
    Example:
        1
       / \
      2   3
     / \ / \
       4
2
  
Returns
-------
serial_tree :  A list of
serialized values
    Example:
[1,2,3,None,4,2,None]

In [4]:
from collections import namedtuple
Node = namedtuple('Node', ("value", "left", "right"))

In [5]:
def serialize_tree(node: Node):
	assert node.value is not None, "nodes must have a value, left and right branches are optional"
	return chain([node.value, 
                  node.left and node.left.value,
                  node.right and node.right.value],
                 chain.from_iterable(map(lambda x: next(x) and False or x,
		                             map(serialize_tree, filter(None, [node.left, node.right])))))\
	       if (node.left or node.right) else chain([node.value])

In [6]:
example_tree = Node(1,
	Node(2, None,
	        Node(4, None, None)),
	Node(3, Node(2, None, None),
	        None)
)

list(serialize_tree(example_tree))


[1, 2, 3, None, 4, 2, None]

## Data Section

This section consists of short exercises in analysis of a
relational health-related database in SQLite, downloadable from here:
https://www.dropbox.com/s/mgu1s93kpjsoyhh/openmrs.db?dl=0

This database is a
processed version of the public data set for a specific instance of a query
tool. The original open MRS data model can be found here:
https://wiki.openmrs.org/display/docs/Data+Model. The solutions should be done
on the data in SQLite database provided. You can convert it to a database flavor
of your choice, but again, explain your choice. 

Key tables include:
- patient
- encounter
- encounter_diagnosis
- lab_result
- diagnosis

Please provide both
the data result as well as any code that was run to obtain the result.

In [7]:
import sqlite3
import pandas

conn = sqlite3.connect("openmrs.db")
patient = pandas.read_sql_query('SELECT * from patient;', conn)
patient.rename(columns={'id':'patient_id'}, inplace=True)
diagnosis = pandas.read_sql_query('SELECT * from diagnosis;', conn)
diagnosis.rename(columns={'id':'diagnosis_id'},inplace=True)
encounter = pandas.read_sql_query('SELECT * from encounter;', conn)
encounter.rename(columns={'id':'encounter_id'},inplace=True)
encounter_diagnosis = pandas.read_sql_query('SELECT * from encounter_diagnosis;', conn)
lab_result = pandas.read_sql_query('SELECT * from lab_result;', conn)
lab_result.rename(columns={'id':'lab_result_id'},inplace=True)
conn.close()

### Data Exercise 1

Provide a list of male patients in the database and the
counts of patients by gender.

In [8]:
patient[~(patient.gender == 'M')].patient_id

0          1
8         24
9         28
10        29
11        33
        ... 
5279    9337
5280    9338
5282    9344
5283    9345
5284    9346
Name: patient_id, Length: 3485, dtype: int64

### Data Exercise 2

Count patients in database
diagnosed with DERMITITIS at an
encounter.

In [9]:
dermatitis = diagnosis[diagnosis.name.str.startswith('DERMATITIS')]['diagnosis_id']
derm_encounter = pandas.merge(encounter_diagnosis, dermatitis, on='diagnosis_id', how='inner')['encounter_id']
derm_patient = pandas.merge(encounter, derm_encounter, on='encounter_id', how='inner')['patient_id']
len(derm_patient.unique())

159

### Data Exercise 3

Provide a list
patients, by MRN, who have had a
CD4 count
of less than 300.

In [10]:
lt300cd4_encounter = lab_result[lab_result.cd4 < 300].encounter_id
lt300cd4_patient = pandas.merge(encounter, lt300cd4_encounter, on='encounter_id', how='inner').patient_id
lt300cd4_mrn = pandas.merge(patient, lt300cd4_patient, on='patient_id', how='inner').mrn.unique()
lt300cd4_mrn

array(['MRN000036', 'MRN000042', 'MRN000048', 'MRN000063', 'MRN000065',
       'MRN000083', 'MRN000100', 'MRN000112', 'MRN000148', 'MRN000156',
       'MRN000205', 'MRN000243', 'MRN000266', 'MRN000272', 'MRN000282',
       'MRN000288', 'MRN000325', 'MRN000335', 'MRN000346', 'MRN000381',
       'MRN000387', 'MRN000391', 'MRN000405', 'MRN000432', 'MRN000454',
       'MRN000461', 'MRN000520', 'MRN000526', 'MRN000550', 'MRN000568',
       'MRN000569', 'MRN000574', 'MRN000584', 'MRN000592', 'MRN000610',
       'MRN000630', 'MRN000637', 'MRN000648', 'MRN000683', 'MRN000686',
       'MRN000692', 'MRN000694', 'MRN000710', 'MRN000750', 'MRN000765',
       'MRN000773', 'MRN000792', 'MRN000797', 'MRN000815', 'MRN000833',
       'MRN000848', 'MRN000869', 'MRN000904', 'MRN000914', 'MRN000915',
       'MRN000916', 'MRN000917', 'MRN000922', 'MRN000950', 'MRN000965',
       'MRN000988', 'MRN001020', 'MRN001029', 'MRN001037', 'MRN001063',
       'MRN001066', 'MRN001103', 'MRN001108', 'MRN001150', 'MRN0

### Data Exercise 4

Count all female patients
above the age of 30 in the
database as of today’s
date

In [11]:
from operator import attrgetter
from datetime import date
thirty_years_ago_today = '-'.join(map(lambda t, delta: str(t-delta), attrgetter('year', 'month', 'day')(date.today()), [30, 0, 0]))
len(patient[(patient.birthdate <= thirty_years_ago_today) & (patient.gender == 'F')])

2894

### Bonus Data Exercise
Describe any potential
concerns with either the
data
itself or the design of the
database.

I'll save the detailed discussion for the
interview!  Could be moved to a more star based schema for analytics flexibility.